In [1]:
import pandas as pd

def generate_bash_script(csv_file_path):
    """
    Reads data from a CSV file and generates a bash script to apply tc rules iteratively.
    """
    try:
        df = pd.read_csv(csv_file_path)
    except FileNotFoundError:
        print(f"Error: File not found: '{csv_file_path}'")
        return
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return

    required_columns = [
        'Sydney_Baseline_Thrpt',
        'Sydney_Baseline_Latency',
        'Sydney_Baseline_BER_QPSK'
    ]
    if not all(col in df.columns for col in required_columns):
        missing = [col for col in required_columns if col not in df.columns]
        print(f"Error: Missing columns: {', '.join(missing)}")
        return

    script = [
        "#!/bin/bash",
        "set -x",
        "",
        "# Load environment settings",
        "if [ ! -f ./utils/settings.sh ]; then",
        "    echo \"Error: settings.sh not found\"",
        "    exit 1",
        "fi",
        "source ./utils/settings.sh",
        "",
    ]

    # Placeholder for calculated router_configs
    router_configs = []

    for i, row in df.iterrows():
        sydney_thrpt = pd.to_numeric(row['Sydney_Baseline_Thrpt'], errors='coerce')
        sydney_latency = pd.to_numeric(row['Sydney_Baseline_Latency'], errors='coerce')
        sydney_ber_qpsk = pd.to_numeric(row['Sydney_Baseline_BER_QPSK'], errors='coerce')
        sydney_loss_pct = sydney_ber_qpsk * 100


        melbourne_thrpt = pd.to_numeric(row['Melbourne_Baseline_Thrpt'], errors='coerce')
        melbourne_latency = pd.to_numeric(row['Melbourne_Baseline_Latency'], errors='coerce')
        melbourne_ber_qpsk = pd.to_numeric(row['Melbourne_Baseline_BER_QPSK'], errors='coerce')
        melbourne_loss_pct = melbourne_ber_qpsk * 100


        thrpt = min(round(float(row['Sydney_Baseline_Thrpt'])), round(float(row['Melbourne_Baseline_Thrpt'])))
        latency = round(float(row['Sydney_Baseline_Latency']) + float(row['Melbourne_Baseline_Latency']))
        ber_qpsk = max(float(row['Sydney_Baseline_BER_QPSK']), float(row['Melbourne_Baseline_BER_QPSK']))

        loss_pct = round(ber_qpsk * 100)

        router_configs.append({
            'thrpt': thrpt,
            'latency': latency,
            'ber_qpsk': ber_qpsk,
            'loss_pct': loss_pct
        })

        if pd.isna(thrpt) or pd.isna(latency) or pd.isna(ber_qpsk):
            script.append(f"# Skipped datapoint {i + 1}: Invalid data")
            continue

        



        
        script += [
            f"# --- Datapoint {i + 1} ---",
            f"echo \"Applying {thrpt} Mbps, {latency} ms, Loss: {loss_pct:.4f}%\"",
            "",
            "# Configure ens37 (delay + dualpi2)",
            "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens37 root 2>/dev/null || true\"",
            f"ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens37 root handle 1: netem delay {latency}ms loss {loss_pct}%\"",
            "ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens37 parent 1:1 handle 2: dualpi2\"",
            "",
            "# Configure ens38 (rate + dualpi2)",
            "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens38 root 2>/dev/null || true\"",
            f"ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens38 root handle 1: tbf rate {thrpt}mbit burst 32kbit latency 100ms\"",
            "ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens38 parent 1:1 handle 2: dualpi2\"",

            "",
            f"echo \"Completed setup for datapoint {i + 1}\"",
            "sleep 30",
            ""
        ]
    
    router_df = pd.DataFrame(router_configs)
    router_df.to_csv("router_configs.csv", index=False)

    script += [
        "# Cleanup",
        "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens37 root 2>/dev/null || true\"",
        "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens38 root 2>/dev/null || true\"",
        "echo \"All datapoints processed.\"",
        "exit 0"
    ]

    print("\n--- Generated Bash Script ---")
    print("\n".join(script))
    print("--- End of Bash Script ---\n")
    print("Save this output as a .sh file, make it executable with `chmod +x script.sh`, and run it.")
    import subprocess

    # Save to file
    script_filename = "apply_tc_rules.sh"
    with open(script_filename, "w") as f:
        f.write("\n".join(script))



if __name__ == "__main__":
    csv_path = "Baseline_Satellite_Australia_Simulation_Log_cleaned_starlink_downlink.csv"
    generate_bash_script(csv_path)
    



--- Generated Bash Script ---
#!/bin/bash
set -x

# Load environment settings
if [ ! -f ./utils/settings.sh ]; then
    echo "Error: settings.sh not found"
    exit 1
fi
source ./utils/settings.sh

# --- Datapoint 1 ---
echo "Applying 0 Mbps, 0 ms, Loss: 22.0000%"

# Configure ens37 (delay + dualpi2)
ssh root@"$router_ipaddr" "sudo tc qdisc del dev ens37 root 2>/dev/null || true"
ssh root@"$router_ipaddr" "sudo tc qdisc add dev ens37 root handle 1: netem delay 0ms loss 22%"
ssh root@"$router_ipaddr" "sudo tc qdisc add dev ens37 parent 1:1 handle 2: dualpi2"

# Configure ens38 (rate + dualpi2)
ssh root@"$router_ipaddr" "sudo tc qdisc del dev ens38 root 2>/dev/null || true"
ssh root@"$router_ipaddr" "sudo tc qdisc add dev ens38 root handle 1: tbf rate 0mbit burst 32kbit latency 100ms"
ssh root@"$router_ipaddr" "sudo tc qdisc add dev ens38 parent 1:1 handle 2: dualpi2"

echo "Completed setup for datapoint 1"
sleep 30

# --- Datapoint 2 ---
echo "Applying 0 Mbps, 0 ms, Loss: 40.0000%"

# 